# Add new columns to a Datafeed using Pandas

In the following notebook https://github.com/cyrbaufr/backtrader_tuto/blob/master/backtrader_tuto/Extending_a_datafeed.ipynb I explained how to add a new column to a csv file and load it as a new line in Backtrader.  
Here, I do something similar except that I use class PandasData instead of GenericCSVData

To use Backtrader, the standard process is to import datafeeds with basic information like prices (OHLC), volumes and open interests. However, I can be useful to add other fields such as:  
- **Indicators**: This way, it is faster to develop and run strategies in Backtrader as the results of the indicators are already available and we do not need to create complex Strategy functions to be computed inside loops.  
- **Fundamental or sentiment data**: anything can be added as extra columns.

Each column added to the DataFrame will be considered as a new 'line' by Backtrader. This is an extension of standard lines which are open, high, low, close, volume, openinterest.

For further comments on this topic, have a look at the following post https://community.backtrader.com/topic/2958/extending-data-feeds-using-pandasdata-to-add-custom-calculated-field

In [1]:
# import libraries
import pandas as pd
import backtrader as bt
import datetime  # For datetime objects

## 1. Create DataFrame with a new 'SMA' column

First step is to create a csv file with a new column. For exemple, let's calculate a 20-day Simple Moving Average (SMA) using Pandas.

In [90]:
# Load csv file into a DataFrame
dataframe = pd.read_csv('data/orcl-1995-2014.csv',
           index_col=0)
dataframe.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-12-24,46.360001,46.709999,46.150002,46.230000,43.488419,10238200
2014-12-26,46.189999,46.500000,46.070000,46.099998,43.366119,6901500
2014-12-29,46.020000,46.090000,45.599998,45.610001,42.905186,9701400
2014-12-30,45.549999,45.660000,45.290001,45.340000,42.651192,9968400
2014-12-31,45.450001,45.560001,44.970001,44.970001,42.303135,13269200


Now let's add a column 'SMA' to calculate a 20-day MA

In [128]:
# Add new column SMA
dataframe['sma20']=dataframe['Close'].rolling(20).mean()
dataframe['sma50']=dataframe['Close'].rolling(50).mean()
dataframe

,Open,High,Low,Close,Volume,SMA_20,SMA_50,sma20,sma50
Date,,,,,,,,,
1995-03-14,2.490741,2.509259,2.462963,2.462963,24872800,2.340741,2.218704,NaN,NaN
1995-03-15,2.481482,2.481482,2.416667,2.425926,24435600,2.353086,2.224877,NaN,NaN
1995-03-16,2.425926,2.462963,2.425926,2.462963,19735200,2.363272,2.231420,NaN,NaN
1995-03-17,2.472222,2.500000,2.444444,2.472222,29170800,2.372994,2.239012,NaN,NaN
1995-03-20,2.481482,2.564815,2.453704,2.490741,20464000,2.384877,2.246481,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2014-12-24,46.360001,46.709999,46.150002,46.230000,10238200,42.571500,40.743600,42.5715,40.7436
2014-12-26,46.189999,46.500000,46.070000,46.099998,6901500,42.783000,40.899600,42.7830,40.8996
2014-12-29,46.020000,46.090000,45.599998,45.610001,9701400,42.943000,41.060600,42.9430,41.0606


In [129]:
# Drop lines with NaNs as SMA is not yet available
dataframe.dropna(inplace=True)
dataframe.head()

,Open,High,Low,Close,Volume,SMA_20,SMA_50,sma20,sma50
Date,,,,,,,,,
1995-05-23,2.500000,2.583333,2.486111,2.583333,34003600,2.369444,2.355556,2.369444,2.355556
1995-05-24,2.712963,2.796296,2.694444,2.787037,121257600,2.401389,2.362037,2.401389,2.362037
1995-05-25,2.805556,2.851852,2.703704,2.740741,56305600,2.422685,2.368333,2.422685,2.368333
1995-05-26,2.740741,2.740741,2.648148,2.675926,33050400,2.443519,2.372593,2.443519,2.372593
1995-05-30,2.685185,2.685185,2.537037,2.537037,64638400,2.459028,2.373889,2.459028,2.373889


In [130]:
# To avoid confusion, we drop column "Adj Close" which is not standard
dataframe.drop(columns=['Adj Close'], inplace=True)

KeyError: "['Adj Close'] not found in axis"

In [131]:
dataframe.head()

,Open,High,Low,Close,Volume,SMA_20,SMA_50,sma20,sma50
Date,,,,,,,,,
1995-05-23,2.500000,2.583333,2.486111,2.583333,34003600,2.369444,2.355556,2.369444,2.355556
1995-05-24,2.712963,2.796296,2.694444,2.787037,121257600,2.401389,2.362037,2.401389,2.362037
1995-05-25,2.805556,2.851852,2.703704,2.740741,56305600,2.422685,2.368333,2.422685,2.368333
1995-05-26,2.740741,2.740741,2.648148,2.675926,33050400,2.443519,2.372593,2.443519,2.372593
1995-05-30,2.685185,2.685185,2.537037,2.537037,64638400,2.459028,2.373889,2.459028,2.373889


In [132]:
# Better to set datetime index to avoid errors
dataframe.index = pd.to_datetime(dataframe.index)

The great advantage of Pandas Vs. CSV files is that DataFrames can be saved using more efficient libraries like pickle or others.

## 2. Create a new trading strategy using new column 'SMA'

We must first create a class inherited from class GenericCSVData that will add a new line called 'sma' to standard lines already available in GenericCSVData ('datetime', 'time', 'open', 'high', 'low', 'close', 'volume', 'openinterest')

In [133]:
from backtrader.feeds import PandasData

class MyPandasData(PandasData):
    """
    Create custom Pandas datafeed to match with custome DataFrame format
    ----
    Here we specify positions of columns OHLCV in 'dataframe' above
    """
    params = (  ('datetime', -1),  # autodetect position otherwise it fails
                ('time', None),  # No time available
                ('Open', 1),
                ('High', 2),
                ('Low', 3),
                ('Close', 4),
                ('Volume', 5),
                ('openinterest', None),  # not available
            )

In [134]:
class MyExtendedPandasDataV1(MyPandasData):
    """
    Here we add new lines to custom Pandas datafeed 'MyPandasData'
    ----
    Strange behaviour... First line must have same col number than Volume.
    Otherwise, this triggers dim error. Why? Is it a bug?
    """
    # Add 2 'sma' lines to the inherited ones from the base class
    lines = ('sma20', 'sma50')

    # add the parameter to the parameters inherited from the base class
    # Volume in MyPandasData has index 5 so sma20 should be 6
    # However, it fails! Code ois correct when sma20 is 5 (??)
    params = (('sma20', 5), ('sma50', 6))


In [135]:
class MyExtendedPandasDataV2(PandasData):
    """
    Here we add new lines to the standard PandasData datafeed
    ----
    Make sure SMA column is number 6 in csv file used!
    Other GenericCSVData standard fields will be recognized
    """

    # Add a 'sma' line to the inherited ones from the base class
    lines = ('sma20', 'sma50')

    # add the parameter to the parameters inherited from the base class
    # openinterest in GenericCSVData has index 7 so sma should be 8
    # However, SMA is column 8 in the csv file used so sma should be 6
    params = (('sma20', 5), ('sma50', 6))


Due to the strange behaviour of line numbers, it is recommended to use the method discussed in this post https://community.backtrader.com/topic/158/how-to-feed-backtrader-alternative-data/9

In [136]:
class CustomDataLoader(PandasData):
    """
    Add any new lines to standard fields coming from a DataFrame
    ------
    Better than the 2 classes above!
    ...
    Here we add columns 'sma20' and 'sma50' as new line
    To avoid confusion and mistakes, preferably rename dataframe columns 
    so they have the same name than Backtrader lines
    ...
    Due to the nature of a dataframe and to look for things in the dataframe by name,
    the class features also a datafields atttribute which names the fields (columns in the dataframe).
    """
    # Name of the new lines in Backtrader
    lines = ('sma20','sma50',)
    # Better to inform Backtrader of missing columns
    params = (
        ('openinterest',None),  #None= column not present
        ('sma20',-1),  # autodetect column 'sma20' in dataframe
        ('sma50',-1)  # autodetect column 'sma50' in dataframe
    )
    # Add new fields to Pandas datafields (name of columns in dataframe)
    datafields = PandasData.datafields + (['sma20','sma50'])

We can create a new strategy that uses the SMA column available in the csv file as an input for a trading strategy. Here the strategy is just to print the level of the Simple Moving Average every day.

In [137]:
class TestStrategy(bt.Strategy):
    """
    Print close price in logs every day
    """
    def __init__(self):
        """ Keep a reference to the close line in the datas[0] dataseries"""
        self.dataopen = self.datas[0].lines.open
        self.datahigh = self.datas[0].lines.high
        self.datalow = self.datas[0].lines.low
        self.dataclose = self.datas[0].lines.close
        self.datavol = self.datas[0].lines.volume
        # Define column SMA of the CSV file as a new line
        self.my_sma1 = self.datas[0].lines.sma20
        self.my_sma2 = self.datas[0].lines.sma50

    def log(self, txt, dt=None):
        """ Logging function for this strategy"""
        dt = dt or self.datas[0].datetime.date(0)
        # print date followed by any message found in 'txt'
        print('%s, %s' % (dt.isoformat(), txt))

    def next(self):
        """ Simply log the closing price of the series from the reference"""
        # output will be of format "date, Close, a float" (i.e. close price)
        # self.log('Open, %.2f,  Close, %.2f' % (self.dataopen[0], self.dataclose[0]))
        self.log('O, %.2f, H, %.2f, L, %.2f, C, %.2f,  V, %.0f, SMA1, %.2f, SMA2, %.2f' % (
            self.dataopen[0],
            self.datahigh[0],
            self.datalow[0],
            self.dataclose[0],
            self.datavol[0],
            self.my_sma1[0],
            self.my_sma2[0])
                )

In [138]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add the strategy (print logs)
    cerebro.addstrategy(TestStrategy)

    # Create a Data Feed
    # data = MyExtendedPandasDataV1(
    data = CustomDataLoader(
        dataname=dataframe,
        # Do not pass values before this date
        fromdate=datetime.datetime(2000, 1, 1),
        # Do not pass values before this date
        todate=datetime.datetime(2000, 12, 31),
        )

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2000-01-03, O, 31.16, H, 31.30, L, 27.91, C, 29.53,  V, 98114800, SMA1, 23.65, SMA2, 18.78
2000-01-04, O, 28.88, H, 29.66, L, 26.25, C, 26.92,  V, 116824800, SMA1, 24.02, SMA2, 19.10
2000-01-05, O, 25.41, H, 26.59, L, 24.00, C, 25.50,  V, 166054000, SMA1, 24.30, SMA2, 19.39
2000-01-06, O, 25.04, H, 26.25, L, 23.67, C, 24.00,  V, 109880000, SMA1, 24.55, SMA2, 19.65
2000-01-07, O, 23.75, H, 25.88, L, 23.39, C, 25.84,  V, 91755600, SMA1, 24.86, SMA2, 19.94
2000-01-10, O, 27.00, H, 29.00, L, 26.38, C, 28.94,  V, 91518000, SMA1, 25.25, SMA2, 20.29
2000-01-11, O, 28.16, H, 28.69, L, 27.38, C, 28.09,  V, 86585200, SMA1, 25.66, SMA2, 20.62
2000-01-12, O, 28.06, H, 28.06, L, 25.92, C, 26.41,  V, 83443600, SMA1, 26.02, SMA2, 20.89
2000-01-13, O, 27.12, H, 27.47, L, 25.88, C, 26.27,  V, 55779200, SMA1, 26.20, SMA2, 21.15
2000-01-14, O, 27.25, H, 27.84, L, 26.19, C, 26.70,  V, 57078000, SMA1, 26.41, SMA2, 21.40
2000-01-18, O, 26.97, H, 28.62, L, 26.41, C, 27.81,

In [104]:
# Compare with values in dataframe
dataframe.loc['2000-01-03':'2000-01-05', :]

,Open,High,Low,Close,Volume,SMA_20,SMA_50
Date,,,,,,,
2000-01-03,31.15625,31.296875,27.90625,29.531250,98114800,23.652344,18.777812
2000-01-04,28.87500,29.656250,26.25000,26.921875,116824800,24.017969,19.095625
2000-01-05,25.40625,26.593750,24.00000,25.500000,166054000,24.302344,19.389063


All good! Values printed inside Backtrader are the same than those available in the dataframe.